In [1]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.1 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
import numpy as np
import io
import os
import pandas as pd
import re
import string
import time
from numpy import random
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense,Flatten,InputLayer,BatchNormalization,Dropout,Input,LayerNormalization
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from google.colab import drive
from google.colab import files
from datasets import load_dataset
from transformers import create_optimizer,DataCollatorForSeq2Seq,DataCollatorForLanguageModeling,BlenderbotTokenizerFast,BlenderbotSmallTokenizerFast,TFBlenderbotForConditionalGeneration



In [3]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d grafstor/simple-dialogs-for-chatbot

  0% 0.00/59.3k [00:00<?, ?B/s]
100% 59.3k/59.3k [00:00<00:00, 89.8MB/s]


In [4]:
!unzip "/content/simple-dialogs-for-chatbot.zip" -d "/content/dataset/"

Archive:  /content/simple-dialogs-for-chatbot.zip
  inflating: /content/dataset/dialogs.txt  


In [5]:
question  =[]
answer = []
with open("/content/dataset/dialogs.txt",'r') as f :
    for line in f :
        line  =  line.split('\t')
        question.append(line[0])
        answer.append(line[1])
print(len(question) == len(answer))

True


In [6]:
question[:5]

['hi, how are you doing?',
 "i'm fine. how about yourself?",
 "i'm pretty good. thanks for asking.",
 'no problem. so how have you been?',
 "i've been great. what about you?"]

In [7]:
answer[:5]

["i'm fine. how about yourself?\n",
 "i'm pretty good. thanks for asking.\n",
 'no problem. so how have you been?\n',
 "i've been great. what about you?\n",
 "i've been good. i'm in school right now.\n"]

In [8]:
answer = [ i.replace("\n","") for i in answer]

In [9]:
answer[:5]

["i'm fine. how about yourself?",
 "i'm pretty good. thanks for asking.",
 'no problem. so how have you been?',
 "i've been great. what about you?",
 "i've been good. i'm in school right now."]

In [10]:
 pd.DataFrame({"question" : question ,"answer":answer}).to_csv('discussion.csv')


In [11]:
dataset = pd.read_csv('/content/discussion.csv')

In [12]:
dataset

,Unnamed: 0,question,answer
0,0,"hi, how are you doing?",i'm fine. how about yourself?
1,1,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
2,2,i'm pretty good. thanks for asking.,no problem. so how have you been?
3,3,no problem. so how have you been?,i've been great. what about you?
4,4,i've been great. what about you?,i've been good. i'm in school right now.
...,...,...,...
3720,3720,that's a good question. maybe it's not old age.,are you right-handed?
3721,3721,are you right-handed?,yes. all my life.
3722,3722,yes. all my life.,you're wearing out your right hand. stop using...
3723,3723,you're wearing out your right hand. stop using...,but i do all my writing with my right hand.


In [13]:
dataset.isna().sum()

Unnamed: 0    0
question      0
answer        0
dtype: int64

In [14]:
dataset[dataset['Unnamed: 0']==447]

,Unnamed: 0,question,answer
447,447,"no, i had no idea.","well, she did, her baby was 8 pounds 6 ounces."


In [15]:
dataset.drop_duplicates()

,Unnamed: 0,question,answer
0,0,"hi, how are you doing?",i'm fine. how about yourself?
1,1,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
2,2,i'm pretty good. thanks for asking.,no problem. so how have you been?
3,3,no problem. so how have you been?,i've been great. what about you?
4,4,i've been great. what about you?,i've been good. i'm in school right now.
...,...,...,...
3720,3720,that's a good question. maybe it's not old age.,are you right-handed?
3721,3721,are you right-handed?,yes. all my life.
3722,3722,yes. all my life.,you're wearing out your right hand. stop using...
3723,3723,you're wearing out your right hand. stop using...,but i do all my writing with my right hand.


In [16]:
model_id="facebook/blenderbot-400M-distill"
tokenizer = BlenderbotTokenizerFast.from_pretrained(model_id,truncation_side="left")

In [17]:
dataset['question'] = "<BOS> " + dataset['question'] + " <EOS>"
dataset['answer'] = "<BOS> " + dataset['answer'] + " <EOS>"


In [18]:
dataset.to_csv('dataset.csv')

In [19]:
filepath="/content/dataset.csv"
dataset = load_dataset('csv', data_files=filepath)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-6e65acc7144f8083/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
def preprocess_function(example):
  return tokenizer(
    example['question'],
    text_target=example['answer'],
    padding='max_length',
    max_length=60,
    truncation=True,)

    


In [21]:
tokenized_dataset=dataset.map(
    preprocess_function,remove_columns=dataset["train"].column_names
)

Map:   0%|          | 0/3725 [00:00<?, ? examples/s]

In [22]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3725
    })
})

In [23]:
model = TFBlenderbotForConditionalGeneration.from_pretrained(model_id)

All model checkpoint layers were used when initializing TFBlenderbotForConditionalGeneration.

Some layers of TFBlenderbotForConditionalGeneration were not initialized from the model checkpoint at facebook/blenderbot-400M-distill and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer,model=model,return_tensors="tf")

In [25]:
print(model.config)


BlenderbotConfig {
  "_name_or_path": "facebook/blenderbot-400M-distill",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "BlenderbotForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1280,
  "decoder_attention_heads": 32,
  "decoder_ffn_dim": 5120,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": true,
  "dropout": 0.1,
  "encoder_attention_heads": 32,
  "encoder_ffn_dim": 5120,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 2,
  "encoder_no_repeat_ngram_size": 3,
  "eos_token_id": 2,
  "extra_layer_norm": false,
  "extra_pos_embeddings": 0,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
 

In [26]:
tf_train_dataset=tokenized_dataset['train'].to_tf_dataset(
    shuffle=True,
    batch_size=4,
    collate_fn=data_collator
)

You're using a BlenderbotTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [27]:
for i in  tf_train_dataset.take(1):
  print(i)

{'input_ids': <tf.Tensor: shape=(4, 60), dtype=int64, numpy=
array([[4440,   41, 3314,   37,  466,  372,  271, 1205, 1347,   38, 4440,
          44, 3314,   37,    2,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0],
       [4440,   41, 3314,   37,  607,  360, 2815,  306, 1329,   21, 4440,
          44, 3314,   37,    2,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0],
       [4440,   41, 3314,   37,  360,  304, 1203,  265, 3495,   38, 4440,
          44, 3314,   37,    2,    0,    0,    

In [28]:
def replacements(a):
  for i in [1,2]:
    condition = tf.equal(a, i)
    case_true = -100*tf.ones_like(a)
    
    case_false = a
    a=tf.where(condition, case_true, case_false)
  return a


In [29]:
a=tf.constant([[1,3,234,445,2,2,2],
               [1,3445,234,34,23,2,2]])
replacements(a)

<tf.Tensor: shape=(2, 7), dtype=int32, numpy=
array([[-100,    3,  234,  445, -100, -100, -100],
       [-100, 3445,  234,   34,   23, -100, -100]], dtype=int32)>

In [30]:
def prepare_labels(inputs):
  return {'input_ids':inputs['input_ids'],
          'attention_mask':inputs['attention_mask'],
          'labels':replacements(inputs['labels'])}

In [31]:
train_dataset=tf_train_dataset.map(prepare_labels)

In [32]:
for i in train_dataset.take(1):
  print(i)

{'input_ids': <tf.Tensor: shape=(4, 60), dtype=int64, numpy=
array([[4440,   41, 3314,   37, 1794,   21,  359,  446,  342,  360,  287,
        2240,  287, 1868,  420, 3914,   21, 4440,   44, 3314,   37,    2,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0],
       [4440,   41, 3314,   37, 3628,   19,  551,   38,  319,  341, 1305,
          21, 4440,   44, 3314,   37,    2,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0],
       [4440,   41, 3314,   37,  271,  487,  319,  264,  457,  276, 1014,
          38, 4440,   44, 3314,   37,    2,    

In [33]:
model.summary()

Model: "tf_blenderbot_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (TFBlenderbotMainLaye  multiple                 364802560 
 r)                                                              
                                                                 
 final_logits_bias (BiasLaye  multiple                 8008      
 r)                                                              
                                                                 
Total params: 364,810,568
Trainable params: 364,802,560
Non-trainable params: 8,008
_________________________________________________________________


In [34]:
num_train_steps=len(tf_train_dataset)
optimizer, schedule = create_optimizer(
  init_lr=6e-5,
  num_warmup_steps=1_000,
  num_train_steps=num_train_steps,
)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [35]:
history=model.fit(train_dataset, epochs=1)

932/932 [==============================] - 350s 268ms/step - loss: 1.5473


In [37]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [38]:


# Save the model weights to a file on Google Drive
model.save_weights('/content/drive/MyDrive/my_model__chatbot_simple_dialogues_weights.h5')

In [39]:
model.load_weights('/content/drive/MyDrive/my_model__chatbot_simple_dialogues_weights.h5')

In [40]:
input_text='hello how are you'

In [41]:
history=tokenizer(input_text, return_tensors="tf")

In [50]:
MAX_NEW_TOKENS=50

In [43]:
init_time=time.time()
output=model.generate(**history,max_new_tokens=MAX_NEW_TOKENS,do_sample=True,top_p=0.9)

print(tokenizer.decode(output[0]))
print(time.time()-init_time)

<s> i'm good. how about you? what do you do for a living? i'm a</s>
18.479989051818848


In [44]:
init_time=time.time()
output_temp = model.generate(**history,max_new_tokens=MAX_NEW_TOKENS,do_sample=True,temperature=1.0, top_k=0)
print(tokenizer.decode(output_temp[0]))
print(time.time()-init_time)

<s> I'm doing well. How about you? Do you have any plans for this evening?</s>
19.382099628448486


In [45]:
init_time=time.time()
output_temp = model.generate(**history,max_new_tokens=MAX_NEW_TOKENS, do_sample=True,temperature=2.0, top_k=0)
print(tokenizer.decode(output_temp[0]))
print(time.time()-init_time)

<s> I'm okay. Are you asking where we are up and what is coming in? I'm</s>
16.381337642669678


In [46]:
init_time=time.time()
output_temp = model.generate(**history,max_new_tokens=MAX_NEW_TOKENS,do_sample=True,temperature=0.5, top_k=0)
print(tokenizer.decode(output_temp[0]))
print(time.time()-init_time)

<s> I'm doing well. How about you? What are you up to? I'm going to</s>
16.475351333618164


In [47]:
init_time=time.time()
output_topk = model.generate(**history,max_new_tokens=MAX_NEW_TOKENS,do_sample=True,temperature=2.0,top_k=50)
print(tokenizer.decode(output_topk[0]))
print(time.time()-init_time)

<s> Not that good. I wish I was a little bit different. I've only just started college</s>
9.223061800003052


In [ ]:
MAX_NEW_TOKENS = 100
history = []

while True:
    user_input = input("You: ")  # Get user input
    init_time = time.time()  # Start timing
    
    # Generate bot response
    input_ids = tokenizer.encode(user_input, return_tensors="tf")
    history.extend(input_ids.numpy().tolist())  # Add user input to history
    output = model.generate(input_ids=input_ids, max_length=len(input_ids[0]) + MAX_NEW_TOKENS, do_sample=True)
    
    # Decode and print bot response
    bot_response = tokenizer.decode(output[0], skip_special_tokens=True,top_p=0.9)
    print("Bot:", bot_response)
    
    print("Time elapsed:", time.time() - init_time)  # Print elapsed time
    
    # Add bot response to the history
    history.extend(output[0].numpy().tolist())

You: hi
Bot:  Hi, how are you? What do you do for a living? I'm an accountant.
Time elapsed: 12.905842781066895
You: i am engineer ! better than acountant
Bot:  <Engineer! what do you mean? what makes you say that?
Time elapsed: 13.397329092025757
You: you are mad ?
Bot:  No, not at all.  I'm happy for you.  Why are you mad?
Time elapsed: 13.086717128753662
